In [1]:
import qiskit
import numpy as np

import json, glob, math

In [2]:
glob.glob("*")

['elephant QFT.ipynb',
 'plot-tools.py',
 'png-tools.py',
 'QHOMER_IQFT.ipynb',
 'QHOMER_Optimizer_Function.ipynb',
 'README.md',
 'Resources',
 'VectorQraphics.py']

In [3]:
def qubits_necessary(n):
    """log2 of n"""
    l2 = math.log2(n)
    assert l2 % 1 == 0, "not an integer power of 2"
    return int(l2)

def normalized(arr):
    """l2-normalization"""
    return arr / np.linalg.norm(arr, 2)

## Data Loading

In [4]:
with open("./Resources/elephant.3.json") as f:
    data = json.load(f)[:-1]
    data = normalized(np.array([complex(a, b) for a, b in data]))
    data = np.round(data, 2)

In [5]:
data

array([-0.34-0.2j , -0.35+0.03j, -0.37+0.21j,  0.25+0.2j ,  0.25+0.34j,
        0.31+0.04j,  0.2 -0.23j, -0.07-0.25j])

## Classical FT

In [6]:
data_ft = normalized(np.fft.fft(data))

In [7]:
data_ft

array([-0.04245614+0.04953216j, -0.18816477-0.01940699j,
        0.07076023+0.13444443j, -0.00916641-0.19257912j,
       -0.14152046+0.03538011j,  0.08202443+0.04063506j,
       -0.01415205-0.02122807j, -0.71966393-0.59285941j])

## State Loading

In [8]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute

In [9]:
def to_statevector(self):
    job = execute(self, Aer.get_backend("statevector_simulator"))
    return job.result().get_statevector()

QuantumCircuit.to_statevector = to_statevector

In [10]:
def QHomer_Init(data_ft):
    """creates an initialization circuit creating a state with data_ft as coefficients"""
    # number of qubits necessary
    qubit_count = qubits_necessary(len(data_ft))
    #print(f"need {qubit_count} qubits")

    # quantum and classical registers
    qregs = QuantumRegister(qubit_count)
    cregs = ClassicalRegister(qubit_count)

    # array of complex amplitudes
    desired_state = data_ft

    # initialization circut
    circuit = QuantumCircuit(qregs, cregs, name="data_loader_trivial")
    circuit.initialize(desired_state, range(qubit_count))
    
    return circuit

## Inverse QFT

In [11]:
def QHomer_IQFT(circuit):
    from qiskit.aqua.components.iqfts.standard import Standard as StandardIQFT

    IQFT = StandardIQFT(len(circuit.qubits))
    iqft = IQFT.construct_circuit(mode="circuit", qubits=circuit.qubits, circuit=circuit)

    return iqft


## Simulate and Measure

In [12]:
from qiskit.circuit import Measure

In [13]:
def QHomer_Measurement(circuit, idx, phase):
    """Measure given register with respect to basis |0> + exp(i phase)|1>"""
    
    qregs, cregs = circuit.qregs, circuit.cregs
    
    circuit.u1(phase, idx)
    circuit.h(idx)
    circuit.measure(idx, idx)
    
    return circuit

In [14]:
def QHomer_Cheap_Mans_Tomography(data_ft):
    simulator = Aer.get_backend("qasm_simulator")
    
    def _Cheap_Mans_Tomography(phase):
        circuit_init = QHomer_Init(data_ft)
        circuit_iqft = QHomer_IQFT(circuit_init)
        
        circuit_all = QHomer_Measurement(circuit_iqft, range(len(circuit_init.qubits)), phase)    
        
        result = execute(circuit_iqft, backend = simulator, shots = 1000).result()
        
        return result.get_counts()
    
    def _pivot_measurement_counts(counts):
        n_qubits = len(list(counts.keys())[0])
        pivoted_counts = np.zeros(n_qubits)
        
        for i in range(n_qubits):
            for key in counts:
                if key[i] == "1":
                    pivoted_counts[i] += counts[key]
                    
        return pivoted_counts
    
    phase_results = {
        phase: _pivot_measurement_counts(
            _Cheap_Mans_Tomography(phase)
        ) for phase in np.arange(0, 2*math.pi, .1)
    }
    
    phases = np.array(list(phase_results.keys()))
    values = np.stack(list(phase_results.values()))
    
    best_phase_indices = np.argmax(values, axis=0)
    return phases[best_phase_indices]


phase_tomography_results = QHomer_Cheap_Mans_Tomography(data_ft)

In [16]:
print(phase_tomography_results)

[6.2 1.3 2.9]


NameError: name 'result' is not defined